# Time Series Predictability Classification Notebook

This notebook allows you to:
1. Query time series from Victoria Metrics using a PromQL selector
2. Evaluate predictability characteristics for each time series
3. Classify each series by predictability for Prophet and ARIMA forecasting models
4. Visualize results including historical data grouped by categories

**Classification Categories:**
- **Predictable**: Series suitable for forecasting (includes those with clear seasonality patterns)
- **Low Predictability**: Weak patterns, high error
- **Not Suitable**: Cannot be forecasted (insufficient data, model failures, etc.)

**Use this notebook for:**
- Understanding which time series are suitable for forecasting
- Comparing Prophet vs ARIMA suitability
- Identifying patterns in predictable vs unpredictable series
- Making informed decisions about which models to use for different metrics


## 1. Configuration and Imports


In [ ]:
# Configuration
import os
import sys
from pathlib import Path

# Add current directory to Python path
current_dir = str(Path.cwd())
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

# Victoria Metrics connection - from environment variables
VM_QUERY_URL = os.getenv('VM_QUERY_URL', 'http://victoria-metrics:8428')
VM_TOKEN = os.getenv('VM_TOKEN', '')

print(f"VM Query URL: {VM_QUERY_URL}")


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
import warnings
warnings.filterwarnings('ignore')

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical tests
from statsmodels.tsa.stattools import adfuller, kpss
from scipy import stats

# Darts imports for time series and models
from darts import TimeSeries
from darts.models import Prophet as DartsProphet, ARIMA as DartsARIMA
from darts.metrics import mape, rmse, mae

# Helper modules
from prometheus_api_client import PrometheusConnect

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 8)

print("Imports successful")


## 2. Connect to Victoria Metrics and Query Data

**Configure your selector and history days in the cell below.**


In [ ]:
# PromQL selector - EDIT THIS
SELECTOR = '{job="extractor"}'  # Your PromQL selector

# History parameter - EDIT THIS
HISTORY_DAYS = 365  # Days of history to fetch

# Cross-validation parameters (optimized for ~7 months of data)
CV_FOLDS = 5  # Number of cross-validation folds
CV_HORIZON = 14  # Forecast horizon for CV (days) - 2 weeks
# Minimum data points: need enough for k-fold (cv_horizon * (cv_folds + 1) + minimum training size)
MIN_HISTORY_POINTS = max(60, CV_HORIZON * (CV_FOLDS + 1))  # At least 60 days for seasonality + k-fold requirements
MAX_SERIES_PER_PLOT = 50  # Maximum series per category plot

# Connect to Victoria Metrics and query historical data
headers = {"Authorization": f"Bearer {VM_TOKEN}"} if VM_TOKEN else {}
prom = PrometheusConnect(url=VM_QUERY_URL, headers=headers, disable_ssl=True)
print(f"Connected to Victoria Metrics at {VM_QUERY_URL}")

print(f"\nQuerying: {SELECTOR}")
end_date = datetime.now(timezone.utc)
start_date = end_date - timedelta(days=HISTORY_DAYS)
query_result = prom.custom_query_range(
    query=SELECTOR.replace("'", '"'),  # Ensure double quotes for PromQL
    start_time=start_date,
    end_time=end_date,
    step="24h"
)

print(f"Query range: {start_date.date()} to {end_date.date()}")
print(f"Query returned {len(query_result)} series")


## 3. Parse and Prepare Time Series Data

This parses ALL time series returned by the selector query.


In [ ]:
# Parse all series from query result
all_series = []
for item in query_result:
    metric = item.get('metric', {})
    metric_name = metric.get('__name__')
    if not metric_name:
        continue
    labels = {k: v for k, v in metric.items() if k != '__name__'}
    values = item.get('values', [])
    samples = [(datetime.fromtimestamp(float(ts), tz=timezone.utc), float(value)) for ts, value in values]
    if samples:
        all_series.append((samples, {'metric_name': metric_name, 'labels': labels}))

if not all_series:
    raise ValueError("No data found for selector")

print(f"Found {len(all_series)} time series for selector: {SELECTOR}")
print("\nSeries preview:")
for idx, (samples, series_info) in enumerate(all_series[:5]):
    print(f"  {idx+1}. {series_info['metric_name']} {series_info['labels']}")
if len(all_series) > 5:
    print(f"  ... and {len(all_series) - 5} more")


## 4. Evaluate Predictability Features

This section computes statistical features and model performance metrics for each time series.


In [ ]:
def compute_statistical_features(df):
    """Compute statistical features for a time series."""
    features = {}
    
    if len(df) < 2:
        return features
    
    values = df['y'].values
    
    # Basic statistics
    features['mean'] = np.mean(values)
    features['std'] = np.std(values)
    features['cv'] = features['std'] / features['mean'] if features['mean'] != 0 else np.inf
    features['min'] = np.min(values)
    features['max'] = np.max(values)
    features['range'] = features['max'] - features['min']
    
    # Stationarity tests
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            adf_result = adfuller(values, autolag='AIC')
        features['adf_pvalue'] = adf_result[1]
        features['adf_statistic'] = adf_result[0]
        features['is_stationary_adf'] = adf_result[1] < 0.05
    except Exception:
        features['adf_pvalue'] = np.nan
        features['adf_statistic'] = np.nan
        features['is_stationary_adf'] = False
    
    try:
        with warnings.catch_warnings():
            # Suppress KPSS warnings about p-values being smaller than returned
            warnings.simplefilter("ignore", UserWarning)
            warnings.simplefilter("ignore", RuntimeWarning)
            kpss_result = kpss(values, regression='ct', nlags='auto')
        features['kpss_pvalue'] = kpss_result[1]
        features['kpss_statistic'] = kpss_result[0]
        # Handle case where p-value might be very small (outside lookup table range)
        if kpss_result[1] is not None and not np.isnan(kpss_result[1]):
            features['is_stationary_kpss'] = kpss_result[1] > 0.05
        else:
            # If p-value is None or NaN, use statistic to infer (very small p-value = non-stationary)
            features['is_stationary_kpss'] = False
    except Exception:
        features['kpss_pvalue'] = np.nan
        features['kpss_statistic'] = np.nan
        features['is_stationary_kpss'] = False
    
    # Trend strength (linear regression R²)
    try:
        x = np.arange(len(values))
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, values)
        features['trend_r2'] = r_value ** 2
        features['trend_slope'] = slope
        features['trend_pvalue'] = p_value
    except Exception:
        features['trend_r2'] = np.nan
        features['trend_slope'] = np.nan
        features['trend_pvalue'] = np.nan
    
    # Autocorrelation at lag 1
    try:
        if len(values) > 1:
            autocorr = np.corrcoef(values[:-1], values[1:])[0, 1]
            features['autocorr_lag1'] = autocorr if not np.isnan(autocorr) else 0
        else:
            features['autocorr_lag1'] = 0
    except Exception:
        features['autocorr_lag1'] = 0
    
    # Variance stability (coefficient of variation)
    features['variance_stability'] = 1.0 / (1.0 + features['cv']) if features['cv'] > 0 else 0
    
    # Seasonality detection
    # Check for weekly seasonality (7-day pattern)
    if len(values) >= 14:  # Need at least 2 weeks
        try:
            # Calculate autocorrelation at lag 7 (weekly)
            if len(values) > 7:
                weekly_autocorr = np.corrcoef(values[:-7], values[7:])[0, 1] if len(values) > 7 else 0
                features['weekly_autocorr'] = weekly_autocorr if not np.isnan(weekly_autocorr) else 0
                features['has_weekly_seasonality'] = abs(weekly_autocorr) > 0.3  # Threshold for weekly pattern
            else:
                features['weekly_autocorr'] = 0
                features['has_weekly_seasonality'] = False
        except Exception:
            features['weekly_autocorr'] = 0
            features['has_weekly_seasonality'] = False
    else:
        features['weekly_autocorr'] = 0
        features['has_weekly_seasonality'] = False
    
    # Check for monthly seasonality (30-day pattern)
    if len(values) >= 60:  # Need at least 2 months
        try:
            lag = min(30, len(values) - 1)
            if len(values) > lag:
                monthly_autocorr = np.corrcoef(values[:-lag], values[lag:])[0, 1] if len(values) > lag else 0
                features['monthly_autocorr'] = monthly_autocorr if not np.isnan(monthly_autocorr) else 0
                features['has_monthly_seasonality'] = abs(monthly_autocorr) > 0.3  # Threshold for monthly pattern
            else:
                features['monthly_autocorr'] = 0
                features['has_monthly_seasonality'] = False
        except Exception:
            features['monthly_autocorr'] = 0
            features['has_monthly_seasonality'] = False
    else:
        features['monthly_autocorr'] = 0
        features['has_monthly_seasonality'] = False
    
    # Check for yearly seasonality (365-day pattern) - only if we have enough data
    if len(values) >= 730:  # Need at least 2 years
        try:
            lag = min(365, len(values) - 1)
            if len(values) > lag:
                yearly_autocorr = np.corrcoef(values[:-lag], values[lag:])[0, 1] if len(values) > lag else 0
                features['yearly_autocorr'] = yearly_autocorr if not np.isnan(yearly_autocorr) else 0
                features['has_yearly_seasonality'] = abs(yearly_autocorr) > 0.3  # Threshold for yearly pattern
            else:
                features['yearly_autocorr'] = 0
                features['has_yearly_seasonality'] = False
        except Exception:
            features['yearly_autocorr'] = 0
            features['has_yearly_seasonality'] = False
    else:
        features['yearly_autocorr'] = 0
        features['has_yearly_seasonality'] = False
    
    return features


In [ ]:
def evaluate_prophet_model(series, cv_folds=5, cv_horizon=7, stat_features=None):
    """Evaluate Prophet model using k-fold walk-forward cross-validation.
    
    Args:
        series: TimeSeries object
        cv_folds: Number of cross-validation folds
        cv_horizon: Forecast horizon for CV
        stat_features: Dictionary of statistical features including seasonality detection
    """
    metrics = {}
    
    try:
        if series is None:
            return {'mape': np.inf, 'rmse': np.inf, 'mae': np.inf, 'success': False, 'error': 'Series is None'}
        
        # Minimum data check: need enough for k-fold with minimum training size
        min_training_size = max(60, cv_horizon * 2)  # At least 60 days or 2x horizon for training
        min_total_size = min_training_size + cv_horizon * cv_folds
        
        if len(series) < min_total_size:
            return {'mape': np.inf, 'rmse': np.inf, 'mae': np.inf, 'success': False, 
                   'error': f'Insufficient data: {len(series)} < {min_total_size} (min for {cv_folds} folds)'}
        
        # Detect seasonality from features if provided
        has_weekly = False
        has_monthly = False
        has_yearly = False
        
        if stat_features:
            has_weekly = stat_features.get('has_weekly_seasonality', False)
            has_monthly = stat_features.get('has_monthly_seasonality', False)
            has_yearly = stat_features.get('has_yearly_seasonality', False)
        
        # K-fold walk-forward validation
        # Use expanding windows: each fold uses more training data
        series_len = len(series)
        fold_mape = []
        fold_rmse = []
        fold_mae = []
        
        # Calculate fold positions - space them evenly across available data
        # Start from minimum training size, end before the last cv_horizon
        available_range = series_len - min_training_size - cv_horizon
        if available_range <= 0:
            return {'mape': np.inf, 'rmse': np.inf, 'mae': np.inf, 'success': False, 
                   'error': 'Insufficient data for k-fold validation'}
        
        # Space folds evenly across available range
        if cv_folds == 1:
            split_points = [min_training_size]
        else:
            step = available_range / (cv_folds - 1) if cv_folds > 1 else available_range
            split_points = [int(min_training_size + i * step) for i in range(cv_folds)]
        
        for fold_idx, train_end in enumerate(split_points):
            try:
                # Ensure we have enough room for test set
                if train_end + cv_horizon > series_len:
                    continue
                
                train_series = series[:train_end]
                test_series = series[train_end:train_end + cv_horizon]
                
                # Build add_seasonalities list for custom seasonalities
                # Monthly must be added via add_seasonalities (Prophet doesn't have built-in monthly)
                add_seasonalities = []
                
                # Add monthly seasonality if detected
                if has_monthly:
                    add_seasonalities.append({
                        'name': 'monthly',
                        'seasonal_periods': 30.5,
                        'fourier_order': 5
                    })
                
                # If weekly is detected, add it as custom seasonality (disable built-in)
                # This matches the pattern used in recommended parameters
                if has_weekly:
                    add_seasonalities.append({
                        'name': 'weekly',
                        'seasonal_periods': 7.0,
                        'fourier_order': 3
                    })
                    # When using custom weekly seasonality, disable built-in weekly
                    weekly_seasonality_param = False
                else:
                    weekly_seasonality_param = False
                
                # Prophet parameters - enable seasonality if detected
                prophet_params = {
                    'yearly_seasonality': has_yearly,
                    'weekly_seasonality': weekly_seasonality_param,
                    'daily_seasonality': False,
                    'seasonality_mode': 'additive',
                    'changepoint_prior_scale': 0.5,
                }
                
                # Add custom seasonalities if any are detected
                if add_seasonalities:
                    prophet_params['add_seasonalities'] = add_seasonalities
                
                model = DartsProphet(**prophet_params)
                
                # Fit and predict
                model.fit(train_series)
                forecast = model.predict(len(test_series))
                
                # Calculate metrics for this fold
                fold_mape.append(mape(test_series, forecast))
                fold_rmse.append(rmse(test_series, forecast))
                fold_mae.append(mae(test_series, forecast))
                
            except Exception as fold_error:
                # Skip this fold if it fails
                continue
        
        # Check if we got any successful folds
        if len(fold_mape) == 0:
            return {'mape': np.inf, 'rmse': np.inf, 'mae': np.inf, 'success': False, 
                   'error': 'All folds failed'}
        
        # Aggregate metrics across folds (mean and std)
        metrics = {
            'mape': np.mean(fold_mape),
            'rmse': np.mean(fold_rmse),
            'mae': np.mean(fold_mae),
            'mape_std': np.std(fold_mape),
            'rmse_std': np.std(fold_rmse),
            'mae_std': np.std(fold_mae),
            'folds_successful': len(fold_mape),
            'folds_total': cv_folds,
            'success': True
        }
        
    except Exception as e:
        metrics = {'mape': np.inf, 'rmse': np.inf, 'mae': np.inf, 'success': False, 'error': str(e)}
    
    return metrics


In [ ]:
def evaluate_arima_model(series, cv_folds=5, cv_horizon=7):
    """Evaluate ARIMA model using k-fold walk-forward cross-validation."""
    metrics = {}
    
    try:
        if series is None:
            return {'mape': np.inf, 'rmse': np.inf, 'mae': np.inf, 'aic': np.inf, 'bic': np.inf, 'success': False, 'error': 'Series is None'}
        
        # Minimum data check: need enough for k-fold with minimum training size
        min_training_size = max(60, cv_horizon * 2)  # At least 60 days or 2x horizon for training
        min_total_size = min_training_size + cv_horizon * cv_folds
        
        if len(series) < min_total_size:
            return {'mape': np.inf, 'rmse': np.inf, 'mae': np.inf, 'aic': np.inf, 'bic': np.inf, 'success': False,
                   'error': f'Insufficient data: {len(series)} < {min_total_size} (min for {cv_folds} folds)'}
        
        # K-fold walk-forward validation
        series_len = len(series)
        fold_mape = []
        fold_rmse = []
        fold_mae = []
        fold_aic = []
        fold_bic = []
        
        # Calculate fold positions - space them evenly across available data
        available_range = series_len - min_training_size - cv_horizon
        if available_range <= 0:
            return {'mape': np.inf, 'rmse': np.inf, 'mae': np.inf, 'aic': np.inf, 'bic': np.inf, 'success': False,
                   'error': 'Insufficient data for k-fold validation'}
        
        # Space folds evenly across available range
        if cv_folds == 1:
            split_points = [min_training_size]
        else:
            step = available_range / (cv_folds - 1) if cv_folds > 1 else available_range
            split_points = [int(min_training_size + i * step) for i in range(cv_folds)]
        
        for fold_idx, train_end in enumerate(split_points):
            try:
                # Ensure we have enough room for test set
                if train_end + cv_horizon > series_len:
                    continue
                
                train_series = series[:train_end]
                test_series = series[train_end:train_end + cv_horizon]
                
                # Default ARIMA(1,1,1) parameters
                model = DartsARIMA(p=1, d=1, q=1)
                model.fit(train_series)
                forecast = model.predict(len(test_series))
                
                # Calculate metrics for this fold
                fold_mape.append(mape(test_series, forecast))
                fold_rmse.append(rmse(test_series, forecast))
                fold_mae.append(mae(test_series, forecast))
                
                # Get AIC/BIC from model if available (use last fold's values)
                try:
                    if hasattr(model, 'model') and model.model is not None:
                        if hasattr(model.model, 'aic'):
                            fold_aic.append(model.model.aic)
                        if hasattr(model.model, 'bic'):
                            fold_bic.append(model.model.bic)
                except (AttributeError, TypeError, ValueError):
                    pass
                
            except Exception as fold_error:
                # Skip this fold if it fails
                continue
        
        # Check if we got any successful folds
        if len(fold_mape) == 0:
            return {'mape': np.inf, 'rmse': np.inf, 'mae': np.inf, 'aic': np.inf, 'bic': np.inf, 'success': False,
                   'error': 'All folds failed'}
        
        # Aggregate metrics across folds
        aic_val = np.mean(fold_aic) if len(fold_aic) > 0 else np.nan
        bic_val = np.mean(fold_bic) if len(fold_bic) > 0 else np.nan
        
        metrics = {
            'mape': np.mean(fold_mape),
            'rmse': np.mean(fold_rmse),
            'mae': np.mean(fold_mae),
            'mape_std': np.std(fold_mape),
            'rmse_std': np.std(fold_rmse),
            'mae_std': np.std(fold_mae),
            'aic': aic_val,
            'bic': bic_val,
            'folds_successful': len(fold_mape),
            'folds_total': cv_folds,
            'success': True
        }
        
    except Exception as e:
        metrics = {'mape': np.inf, 'rmse': np.inf, 'mae': np.inf, 'aic': np.inf, 'bic': np.inf, 'success': False, 'error': str(e)}
    
    return metrics


In [ ]:
# Process each series and evaluate predictability
evaluation_results = []

print(f"Evaluating {len(all_series)} time series...")
print(f"{'='*60}")

for series_idx, (samples, series_info) in enumerate(all_series):
    print(f"\nProcessing {series_idx + 1}/{len(all_series)}: {series_info['metric_name']}")
    
    try:
        # Prepare data
        df = pd.DataFrame(samples, columns=['ds', 'y'])
        df['ds'] = pd.to_datetime(df['ds'], utc=True).dt.tz_localize(None)
        df = df.sort_values('ds').reset_index(drop=True)
        
        # Check minimum data points
        if len(df) < MIN_HISTORY_POINTS:
            print(f"  ⚠️  Skipping: insufficient data ({len(df)} < {MIN_HISTORY_POINTS})")
            result = {
                'series_info': series_info,
                'df': df,
                'statistical_features': {},
                'prophet_metrics': {'success': False},
                'arima_metrics': {'success': False},
                'classification': 'Not Suitable',
                'reason': 'Insufficient data'
            }
            evaluation_results.append(result)
            continue
        
        # Check for invalid values
        if df['y'].isna().all() or np.isinf(df['y']).any():
            print(f"  ⚠️  Skipping: invalid values")
            result = {
                'series_info': series_info,
                'df': df,
                'statistical_features': {},
                'prophet_metrics': {'success': False},
                'arima_metrics': {'success': False},
                'classification': 'Not Suitable',
                'reason': 'Invalid values'
            }
            evaluation_results.append(result)
            continue
        
        # Compute statistical features
        print(f"  Computing statistical features...")
        stat_features = compute_statistical_features(df)
        
        # Convert to darts TimeSeries with daily frequency, handling missing dates
        # Create complete date range with daily frequency
        date_range = pd.date_range(start=df['ds'].min(), end=df['ds'].max(), freq='D')
        
        # Set index and reindex to fill missing dates
        df_indexed = df.set_index('ds')[['y']]
        df_complete = df_indexed.reindex(date_range)
        
        # Forward fill missing values, then backward fill any remaining
        df_complete['y'] = df_complete['y'].ffill().bfill()
        
        # Convert to darts TimeSeries
        series = TimeSeries.from_dataframe(df_complete)
        
        # Evaluate Prophet model (with seasonality if detected)
        print(f"  Evaluating Prophet model...")
        prophet_metrics = evaluate_prophet_model(series, cv_folds=CV_FOLDS, cv_horizon=CV_HORIZON, stat_features=stat_features)
        
        # Evaluate ARIMA model
        print(f"  Evaluating ARIMA model...")
        arima_metrics = evaluate_arima_model(series, cv_folds=CV_FOLDS, cv_horizon=CV_HORIZON)
        
        result = {
            'series_info': series_info,
            'df': df,
            'statistical_features': stat_features,
            'prophet_metrics': prophet_metrics,
            'arima_metrics': arima_metrics,
        }
        
        evaluation_results.append(result)
        print(f"  ✓ Completed")
        
    except Exception as exc:
        print(f"  ✗ Failed: {exc}")
        result = {
            'series_info': series_info,
            'df': pd.DataFrame(),
            'statistical_features': {},
            'prophet_metrics': {'success': False, 'error': str(exc)},
            'arima_metrics': {'success': False, 'error': str(exc)},
            'classification': 'Not Suitable',
            'reason': f'Error: {str(exc)}'
        }
        evaluation_results.append(result)
        continue

print(f"\n{'='*60}")
print(f"Evaluation complete: {len(evaluation_results)} series processed")


## 5. Classify Series by Predictability

Classify each series into one of three categories based on model performance, statistical features, and seasonality detection.

**Seasonality Boost**: Series with detected seasonal patterns (weekly, monthly, yearly) are considered more predictable. The classification thresholds are relaxed when seasonality is detected:
- **1 seasonality pattern**: Thresholds relaxed by ~3% (MAPE) and ~5% (RMSE)
- **2+ seasonality patterns**: Thresholds relaxed by ~5% (MAPE) and ~10% (RMSE)

This means a series with clear seasonal patterns can be classified as "Predictable" even if its error metrics are slightly higher than the standard thresholds.


In [ ]:
def classify_predictability(metrics, stat_features, mean_value):
    """Classify time series predictability based on metrics, features, and seasonality detection.
    
    Seasonality detection can boost the classification - series with clear seasonal patterns
    are more predictable and may be upgraded to a higher category.
    """
    # Handle None values
    if metrics is None:
        return 'Not Suitable', 'Metrics are None'
    
    if stat_features is None:
        stat_features = {}
    
    if not metrics.get('success', False):
        return 'Not Suitable', 'Model training failed'
    
    mape_val = metrics.get('mape', np.inf)
    rmse_val = metrics.get('rmse', np.inf)
    
    # Calculate relative RMSE (as percentage of mean)
    if mean_value is None or mean_value <= 0:
        rmse_pct = np.inf
    else:
        rmse_pct = (rmse_val / mean_value) * 100
    
    # Check for seasonality patterns (boosts predictability)
    has_weekly = stat_features.get('has_weekly_seasonality', False) if isinstance(stat_features, dict) else False
    has_monthly = stat_features.get('has_monthly_seasonality', False) if isinstance(stat_features, dict) else False
    has_yearly = stat_features.get('has_yearly_seasonality', False) if isinstance(stat_features, dict) else False
    has_seasonality = has_weekly or has_monthly or has_yearly
    
    # Count number of seasonality patterns detected
    seasonality_count = sum([has_weekly, has_monthly, has_yearly])
    
    # Seasonality boost: adjust thresholds if seasonality is detected
    # Strong seasonality (2+ patterns) provides more boost
    if seasonality_count >= 2:
        # Strong seasonality: relax thresholds by 5% for MAPE and 10% for RMSE
        mape_threshold_high = 15  # instead of 10
        mape_threshold_mod = 35   # instead of 30
        rmse_threshold_high = 30  # instead of 20
        rmse_threshold_mod = 60   # instead of 50
        seasonality_boost = "strong"
    elif seasonality_count == 1:
        # Moderate seasonality: relax thresholds by 3% for MAPE and 5% for RMSE
        mape_threshold_high = 13  # instead of 10
        mape_threshold_mod = 33   # instead of 30
        rmse_threshold_high = 25  # instead of 20
        rmse_threshold_mod = 55   # instead of 50
        seasonality_boost = "moderate"
    else:
        # No seasonality: use standard thresholds
        mape_threshold_high = 10
        mape_threshold_mod = 30
        rmse_threshold_high = 20
        rmse_threshold_mod = 50
        seasonality_boost = "none"
    
    # Classification logic with seasonality-adjusted thresholds
    # Combined "Predictable" category (formerly Highly + Moderately Predictable)
    # If series meets moderate threshold, it's considered Predictable
    if mape_val < mape_threshold_mod and rmse_pct < rmse_threshold_mod:
        category = 'Predictable'
        reason = f'MAPE={mape_val:.2f}%, RMSE={rmse_pct:.2f}%'
        if has_seasonality:
            seasonality_info = []
            if has_weekly:
                seasonality_info.append('weekly')
            if has_monthly:
                seasonality_info.append('monthly')
            if has_yearly:
                seasonality_info.append('yearly')
            reason += f' (seasonality: {", ".join(seasonality_info)})'
        return category, reason
    
    # Special case: If seasonality is detected and error isn't extremely high, 
    # classify as "Predictable" (seasonality makes series more predictable)
    # This check happens before Low Predictability to give seasonality a boost
    if has_seasonality and seasonality_count >= 1:
        # For series with seasonality, use more lenient thresholds
        # If MAPE < 50% and RMSE < 80%, consider it predictable
        if mape_val < 50.0 and rmse_pct < 80.0:
            category = 'Predictable'
            seasonality_info = []
            if has_weekly:
                seasonality_info.append('weekly')
            if has_monthly:
                seasonality_info.append('monthly')
            if has_yearly:
                seasonality_info.append('yearly')
            reason = f'MAPE={mape_val:.2f}%, RMSE={rmse_pct:.2f}% (seasonality detected: {", ".join(seasonality_info)})'
            return category, reason
    
    # If we get here, check for Low Predictability
    if mape_val < np.inf and rmse_pct < np.inf:
        category = 'Low Predictability'
        reason = f'MAPE={mape_val:.2f}%, RMSE={rmse_pct:.2f}%'
        if has_seasonality:
            # Even with seasonality, if metrics are extremely poor, it's still low predictability
            seasonality_info = []
            if has_weekly:
                seasonality_info.append('weekly')
            if has_monthly:
                seasonality_info.append('monthly')
            if has_yearly:
                seasonality_info.append('yearly')
            reason += f' (has seasonality: {", ".join(seasonality_info)} but very high error)'
        return category, reason
    
    # Final fallback - should not normally reach here, but ensures we always return
    return 'Not Suitable', 'Metrics indicate unsuitability'


In [ ]:
# Classify each series for both Prophet and ARIMA
for result in evaluation_results:
    stat_features = result.get('statistical_features', {})
    mean_value = stat_features.get('mean', 0)
    
    # Classify for Prophet
    prophet_metrics = result.get('prophet_metrics', {})
    prophet_class, prophet_reason = classify_predictability(prophet_metrics, stat_features, mean_value)
    result['prophet_classification'] = prophet_class
    result['prophet_reason'] = prophet_reason
    
    # Classify for ARIMA
    arima_metrics = result.get('arima_metrics', {})
    arima_class, arima_reason = classify_predictability(arima_metrics, stat_features, mean_value)
    result['arima_classification'] = arima_class
    result['arima_reason'] = arima_reason

print("Classification complete!")
print(f"\nSummary:")
print(f"  Total series: {len(evaluation_results)}")

# Count classifications
prophet_counts = {}
arima_counts = {}
for result in evaluation_results:
    p_class = result.get('prophet_classification', 'Unknown')
    a_class = result.get('arima_classification', 'Unknown')
    prophet_counts[p_class] = prophet_counts.get(p_class, 0) + 1
    arima_counts[a_class] = arima_counts.get(a_class, 0) + 1

print(f"\nProphet Classifications:")
for cls, count in sorted(prophet_counts.items()):
    print(f"  {cls}: {count}")

print(f"\nARIMA Classifications:")
for cls, count in sorted(arima_counts.items()):
    print(f"  {cls}: {count}")

# Verify seasonality detection is being used in classification
print(f"\n{'='*60}")
print("SEASONALITY DETECTION VERIFICATION")
print(f"{'='*60}")

# Count series with seasonality patterns
series_with_seasonality = 0
seasonality_by_category = {
    'Predictable': {'with_seasonality': 0, 'total': 0},
    'Low Predictability': {'with_seasonality': 0, 'total': 0},
    'Not Suitable': {'with_seasonality': 0, 'total': 0}
}

for result in evaluation_results:
    stat_features = result.get('statistical_features', {})
    has_weekly = stat_features.get('has_weekly_seasonality', False)
    has_monthly = stat_features.get('has_monthly_seasonality', False)
    has_yearly = stat_features.get('has_yearly_seasonality', False)
    has_seasonality = has_weekly or has_monthly or has_yearly
    
    if has_seasonality:
        series_with_seasonality += 1
    
    # Check Prophet classification
    prophet_class = result.get('prophet_classification', 'Unknown')
    if prophet_class in seasonality_by_category:
        seasonality_by_category[prophet_class]['total'] += 1
        if has_seasonality:
            seasonality_by_category[prophet_class]['with_seasonality'] += 1

print(f"\nTotal series with detected seasonality: {series_with_seasonality}/{len(evaluation_results)}")
print(f"\nSeasonality impact on Prophet classifications:")
for category, stats in seasonality_by_category.items():
    if stats['total'] > 0:
        pct = (stats['with_seasonality'] / stats['total']) * 100
        print(f"  {category}: {stats['with_seasonality']}/{stats['total']} ({pct:.1f}%) have seasonality")

# Show examples of series where seasonality boosted classification
print(f"\nExamples where seasonality was detected:")
example_count = 0
for result in evaluation_results:
    if example_count >= 5:
        break
    stat_features = result.get('statistical_features', {})
    has_weekly = stat_features.get('has_weekly_seasonality', False)
    has_monthly = stat_features.get('has_monthly_seasonality', False)
    has_yearly = stat_features.get('has_yearly_seasonality', False)
    
    if has_weekly or has_monthly or has_yearly:
        series_info = result.get('series_info', {})
        prophet_class = result.get('prophet_classification', 'Unknown')
        prophet_reason = result.get('prophet_reason', '')
        
        seasonality_types = []
        if has_weekly:
            seasonality_types.append('weekly')
        if has_monthly:
            seasonality_types.append('monthly')
        if has_yearly:
            seasonality_types.append('yearly')
        
        print(f"  - {series_info.get('metric_name', 'Unknown')}: {prophet_class}")
        print(f"    Seasonality: {', '.join(seasonality_types)}")
        print(f"    Reason: {prophet_reason}")
        example_count += 1

print(f"\n{'='*60}")


## 6. Visualize Results and Summary

Create summary tables and visualizations of the classification results.


In [ ]:
# Create summary DataFrame with seasonality information
summary_data = []
for result in evaluation_results:
    series_info = result['series_info']
    stat_features = result.get('statistical_features', {})
    prophet_metrics = result.get('prophet_metrics', {})
    arima_metrics = result.get('arima_metrics', {})
    
    # Extract seasonality information
    has_weekly = stat_features.get('has_weekly_seasonality', False)
    has_monthly = stat_features.get('has_monthly_seasonality', False)
    has_yearly = stat_features.get('has_yearly_seasonality', False)
    
    # Create seasonality summary string
    seasonality_list = []
    if has_weekly:
        seasonality_list.append('weekly')
    if has_monthly:
        seasonality_list.append('monthly')
    if has_yearly:
        seasonality_list.append('yearly')
    seasonality_detected = ', '.join(seasonality_list) if seasonality_list else 'None'
    
    summary_data.append({
        'metric_name': series_info['metric_name'],
        'labels': str(series_info.get('labels', {})),
        'data_points': len(result.get('df', pd.DataFrame())),
        'mean': stat_features.get('mean', np.nan),
        'std': stat_features.get('std', np.nan),
        'cv': stat_features.get('cv', np.nan),
        'is_stationary_adf': stat_features.get('is_stationary_adf', False),
        'trend_r2': stat_features.get('trend_r2', np.nan),
        'has_weekly_seasonality': has_weekly,
        'has_monthly_seasonality': has_monthly,
        'has_yearly_seasonality': has_yearly,
        'seasonality_detected': seasonality_detected,
        'prophet_mape': prophet_metrics.get('mape', np.nan),
        'prophet_mape_std': prophet_metrics.get('mape_std', np.nan),
        'prophet_rmse': prophet_metrics.get('rmse', np.nan),
        'prophet_rmse_std': prophet_metrics.get('rmse_std', np.nan),
        'prophet_folds': prophet_metrics.get('folds_successful', 0),
        'prophet_classification': result.get('prophet_classification', 'Unknown'),
        'arima_mape': arima_metrics.get('mape', np.nan),
        'arima_mape_std': arima_metrics.get('mape_std', np.nan),
        'arima_rmse': arima_metrics.get('rmse', np.nan),
        'arima_rmse_std': arima_metrics.get('rmse_std', np.nan),
        'arima_folds': arima_metrics.get('folds_successful', 0),
        'arima_classification': result.get('arima_classification', 'Unknown'),
    })

summary_df = pd.DataFrame(summary_data)

# Display summary table in dataset viewer (not as text)
print("Classification Summary Table:")
print(f"Total series: {len(summary_df)}")
print("\nUse the interactive dataset viewer below to explore, sort, and filter the data.")
print("="*120)

# Display the dataframe - this will show in Jupyter's dataset viewer
from IPython.display import display
display(summary_df)


In [ ]:
# Plot distribution of classifications as pie charts
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Color scheme for categories
colors = {
    'Predictable': '#2ecc71',      # Green
    'Low Predictability': '#f39c12',      # Orange
    'Not Suitable': '#e74c3c'            # Red
}

# Prophet classifications
prophet_counts = summary_df['prophet_classification'].value_counts()
prophet_colors = [colors.get(cat, '#95a5a6') for cat in prophet_counts.index]
axes[0].pie(prophet_counts.values, labels=prophet_counts.index, autopct='%1.1f%%',
            colors=prophet_colors, startangle=90, textprops={'fontsize': 11})
axes[0].set_title('Prophet Classifications', fontsize=14, fontweight='bold', pad=20)

# ARIMA classifications
arima_counts = summary_df['arima_classification'].value_counts()
arima_colors = [colors.get(cat, '#95a5a6') for cat in arima_counts.index]
axes[1].pie(arima_counts.values, labels=arima_counts.index, autopct='%1.1f%%',
            colors=arima_colors, startangle=90, textprops={'fontsize': 11})
axes[1].set_title('ARIMA Classifications', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()


In [ ]:
# Scatter plot: MAPE vs Stationarity
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Prophet
prophet_valid = summary_df[summary_df['prophet_mape'] < np.inf]
if len(prophet_valid) > 0:
    scatter = axes[0].scatter(
        prophet_valid['is_stationary_adf'].astype(int),
        prophet_valid['prophet_mape'],
        c=prophet_valid['trend_r2'],
        s=100,
        alpha=0.6,
        cmap='viridis'
    )
    axes[0].set_xlabel('Stationary (ADF test: 0=No, 1=Yes)', fontsize=12)
    axes[0].set_ylabel('Prophet MAPE (%)', fontsize=12)
    axes[0].set_title('Prophet MAPE vs Stationarity (colored by Trend R²)', fontsize=14, fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=axes[0], label='Trend R²')

# ARIMA
arima_valid = summary_df[summary_df['arima_mape'] < np.inf]
if len(arima_valid) > 0:
    scatter = axes[1].scatter(
        arima_valid['is_stationary_adf'].astype(int),
        arima_valid['arima_mape'],
        c=arima_valid['trend_r2'],
        s=100,
        alpha=0.6,
        cmap='viridis'
    )
    axes[1].set_xlabel('Stationary (ADF test: 0=No, 1=Yes)', fontsize=12)
    axes[1].set_ylabel('ARIMA MAPE (%)', fontsize=12)
    axes[1].set_title('ARIMA MAPE vs Stationarity (colored by Trend R²)', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=axes[1], label='Trend R²')

plt.tight_layout()
plt.show()


In [ ]:
# Model comparison: Prophet vs ARIMA suitability
comparison_data = []
for result in evaluation_results:
    comparison_data.append({
        'metric_name': result['series_info']['metric_name'],
        'prophet_class': result.get('prophet_classification', 'Unknown'),
        'arima_class': result.get('arima_classification', 'Unknown'),
        'prophet_mape': result.get('prophet_metrics', {}).get('mape', np.nan),
        'arima_mape': result.get('arima_metrics', {}).get('mape', np.nan),
    })

comparison_df = pd.DataFrame(comparison_data)

# Create comparison plot
fig, ax = plt.subplots(figsize=(12, 8))

# Filter valid data
valid_comparison = comparison_df[
    (comparison_df['prophet_mape'] < np.inf) & 
    (comparison_df['arima_mape'] < np.inf)
]

if len(valid_comparison) > 0:
    ax.scatter(valid_comparison['prophet_mape'], valid_comparison['arima_mape'], 
               alpha=0.6, s=100)
    
    # Add diagonal line (y=x)
    max_val = max(valid_comparison['prophet_mape'].max(), valid_comparison['arima_mape'].max())
    ax.plot([0, max_val], [0, max_val], 'r--', alpha=0.5, label='Equal performance')
    
    ax.set_xlabel('Prophet MAPE (%)', fontsize=12)
    ax.set_ylabel('ARIMA MAPE (%)', fontsize=12)
    ax.set_title('Model Comparison: Prophet vs ARIMA MAPE', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    # Add quadrant labels
    ax.text(0.95, 0.05, 'Prophet Better', transform=ax.transAxes, 
            ha='right', va='bottom', fontsize=10, style='italic')
    ax.text(0.05, 0.95, 'ARIMA Better', transform=ax.transAxes, 
            ha='left', va='top', fontsize=10, style='italic')

plt.tight_layout()
plt.show()


## 7. Plot Historical Data Grouped by Categories

Visualize historical time series data grouped by their predictability classifications. This allows you to visually inspect the characteristics of series within each category.


In [ ]:
# Generate Prophet parameter recommendations for each category
# This will be used when plotting to show recommended parameters alongside summary statistics
# Note: This tries to use get_recommended_prophet_params if available (Section 9), 
# but has a complete inline fallback if the function isn't defined yet

def generate_prophet_params_for_plotting(evaluation_results):
    """Generate Prophet parameter recommendations for each category.
    
    Returns a dictionary mapping category names to their recommended parameters.
    """
    categories = ['Predictable', 'Low Predictability', 'Not Suitable']
    prophet_params_dict = {}
    
    for category in categories:
        # Get all series in this category
        category_series = [r for r in evaluation_results 
                          if r.get('prophet_classification') == category]
        
        if len(category_series) == 0:
            # No series in this category, use defaults (inline version)
            base_params = {
                'yearly_seasonality': False,
                'weekly_seasonality': False,
                'daily_seasonality': False,
                'seasonality_mode': 'additive',
            }
            if category == 'Predictable':
                base_params.update({
                    'changepoint_prior_scale': 0.5,
                    'seasonality_prior_scale': 10.0,
                    'holidays_prior_scale': 10.0,
                    'mcmc_samples': 0,
                    'interval_width': 0.80,
                    'uncertainty_samples': 1000,
                })
            elif category == 'Low Predictability':
                base_params.update({
                    'changepoint_prior_scale': 0.05,
                    'seasonality_prior_scale': 0.01,
                    'holidays_prior_scale': 10.0,
                    'mcmc_samples': 0,
                    'interval_width': 0.95,
                    'uncertainty_samples': 1000,
                })
            else:  # Not Suitable
                base_params.update({
                    'changepoint_prior_scale': 0.1,
                    'seasonality_prior_scale': 0.01,
                    'holidays_prior_scale': 10.0,
                    'mcmc_samples': 0,
                    'interval_width': 0.95,
                    'uncertainty_samples': 1000,
                })
            prophet_params_dict[category] = base_params
            continue
        
        # Analyze seasonality patterns in this category
        weekly_count = sum(1 for r in category_series 
                          if r.get('statistical_features', {}).get('has_weekly_seasonality', False))
        monthly_count = sum(1 for r in category_series 
                           if r.get('statistical_features', {}).get('has_monthly_seasonality', False))
        yearly_count = sum(1 for r in category_series 
                          if r.get('statistical_features', {}).get('has_yearly_seasonality', False))
        
        # Determine if we should enable seasonality (if >=20% of series show it, or at least 1 series)
        weekly_threshold = max(1, int(len(category_series) * 0.2))
        monthly_threshold = max(1, int(len(category_series) * 0.2))
        yearly_threshold = max(1, int(len(category_series) * 0.2))
        
        enable_weekly = weekly_count >= weekly_threshold
        enable_monthly = monthly_count >= monthly_threshold
        enable_yearly = yearly_count >= yearly_threshold
        
        # Verification: Ensure monthly is enabled if ANY series in category has it
        # (This is a safeguard - the threshold should already handle this, but double-check)
        if monthly_count > 0 and not enable_monthly:
            # If we have monthly seasonality but threshold wasn't met, enable it anyway
            # (This shouldn't happen with threshold=max(1, ...), but just in case)
            enable_monthly = True
        
        # Get base parameters for this category (try to use function if available, otherwise inline)
        try:
            base_params = get_recommended_prophet_params(category).copy()
            # Remove any existing add_seasonalities from the function result - we'll add our own
            if 'add_seasonalities' in base_params:
                del base_params['add_seasonalities']
        except NameError:
            # Function not defined yet, use inline version
            base_params = {
                'yearly_seasonality': False,  # Will be set later based on detection
                'weekly_seasonality': False,  # Will be set later based on detection
                'daily_seasonality': False,
                'seasonality_mode': 'additive',
            }
            if category == 'Predictable':
                base_params.update({
                    'changepoint_prior_scale': 0.5,
                    'seasonality_prior_scale': 10.0,
                    'holidays_prior_scale': 10.0,
                    'mcmc_samples': 0,
                    'interval_width': 0.80,
                    'uncertainty_samples': 1000,
                })
            elif category == 'Low Predictability':
                base_params.update({
                    'changepoint_prior_scale': 0.05,
                    'seasonality_prior_scale': 0.01,
                    'holidays_prior_scale': 10.0,
                    'mcmc_samples': 0,
                    'interval_width': 0.95,
                    'uncertainty_samples': 1000,
                })
            else:  # Not Suitable
                base_params.update({
                    'changepoint_prior_scale': 0.1,
                    'seasonality_prior_scale': 0.01,
                    'holidays_prior_scale': 10.0,
                    'mcmc_samples': 0,
                    'interval_width': 0.95,
                    'uncertainty_samples': 1000,
                })
        
        # Add custom seasonalities - ALWAYS build this list from detected patterns
        # IMPORTANT: Add monthly FIRST, then weekly, to ensure proper ordering
        add_seasonalities = []
        
        # Add monthly seasonality if detected (ALWAYS add when detected)
        # Monthly must be added via add_seasonalities (Prophet doesn't have built-in monthly)
        if enable_monthly:
            add_seasonalities.append({
                'name': 'monthly',
                'seasonal_periods': 30.5,
                'fourier_order': 5
            })
        
        # If weekly is detected, add it as custom seasonality
        if enable_weekly:
            add_seasonalities.append({
                'name': 'weekly',
                'seasonal_periods': 7.0,
                'fourier_order': 3
            })
            # When using custom weekly seasonality, disable built-in weekly
            base_params['weekly_seasonality'] = False
        else:
            # No weekly detected, use built-in if needed (but we're not enabling it here)
            base_params['weekly_seasonality'] = False
        
        # Set yearly seasonality (use built-in, not custom)
        base_params['yearly_seasonality'] = enable_yearly
        
        # ALWAYS add seasonalities to params dictionary if we have any
        # This ensures monthly (and weekly if detected) are always included
        # Even if only one is detected, the list should contain it
        if add_seasonalities:
            base_params['add_seasonalities'] = add_seasonalities
        # If no custom seasonalities, ensure add_seasonalities is not in the dict
        # (to avoid confusion, but this shouldn't happen if monthly/weekly are detected)
        
        prophet_params_dict[category] = base_params
    
    return prophet_params_dict

# Generate parameters for plotting
prophet_params_for_plotting = generate_prophet_params_for_plotting(evaluation_results)


In [ ]:
def plot_series_by_category(evaluation_results, model_type='prophet', max_series_per_plot=20, prophet_params=None):
    """Plot time series grouped by classification category.
    
    Args:
        evaluation_results: List of evaluation results
        model_type: 'prophet' or 'arima'
        max_series_per_plot: Maximum number of series to show per plot
        prophet_params: Optional dict mapping category names to recommended Prophet parameters
    """
    
    # Group series by classification
    categories = {
        'Predictable': [],
        'Low Predictability': [],
        'Not Suitable': []
    }
    
    for result in evaluation_results:
        classification = result.get(f'{model_type}_classification', 'Not Suitable')
        if classification in categories:
            categories[classification].append(result)
    
    # Plot each category
    for category_name, series_list in categories.items():
        if len(series_list) == 0:
            continue
        
        # Limit number of series per plot
        series_to_plot = series_list[:max_series_per_plot]
        n_series = len(series_to_plot)
        
        if n_series == 0:
            continue
        
        # Calculate grid dimensions
        n_cols = min(4, n_series)
        n_rows = (n_series + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5 * n_rows))
        
        # Convert axes to a flat list of Axes objects
        # plt.subplots can return: single Axes, 1D array, 2D array, or list
        if n_rows == 1 and n_cols == 1:
            axes = [axes]
        elif isinstance(axes, np.ndarray):
            axes = axes.flatten().tolist()
        elif not isinstance(axes, list):
            axes = [axes]
        
        # Set suptitle with padding to avoid overlap with subplot titles
        fig.suptitle(f'{model_type.upper()} - {category_name} ({len(series_list)} total, showing {n_series})', 
                     fontsize=16, fontweight='bold', y=1)
        
        for idx, result in enumerate(series_to_plot):
            ax = axes[idx]
            df = result.get('df', pd.DataFrame())
            
            if len(df) == 0:
                ax.text(0.5, 0.5, 'No data', ha='center', va='center', transform=ax.transAxes)
                ax.set_title('No data', fontsize=10)
                continue
            
            # Plot time series
            ax.plot(df['ds'], df['y'], 'b-', linewidth=1.5, alpha=0.7)
            
            # Title with key info
            series_info = result['series_info']
            title = f"{series_info['metric_name']}"
            if series_info.get('labels'):
                # Show first few labels
                label_str = str(series_info['labels'])[:50]
                if len(str(series_info['labels'])) > 50:
                    label_str += '...'
                title += f"\n{label_str}"
            
            # Add metrics to title
            metrics = result.get(f'{model_type}_metrics', {})
            if metrics.get('success', False):
                mape_val = metrics.get('mape', np.nan)
                rmse_val = metrics.get('rmse', np.nan)
                title += f"\nMAPE: {mape_val:.2f}%, RMSE: {rmse_val:.2f}"
            
            ax.set_title(title, fontsize=9)
            ax.set_xlabel('Date', fontsize=8)
            ax.set_ylabel('Value', fontsize=8)
            ax.grid(True, alpha=0.3)
            ax.tick_params(labelsize=7)
        
        # Hide unused subplots
        for idx in range(n_series, len(axes)):
            axes[idx].set_visible(False)
        
        # Adjust layout to prevent suptitle from overlapping with subplot titles
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.show()
        
        # Print summary statistics for this category
        if len(series_list) > 0:
            print(f"\n{category_name} Summary Statistics:")
            mape_values = []
            rmse_values = []
            weekly_count = 0
            monthly_count = 0
            yearly_count = 0
            
            for result in series_list:
                metrics = result.get(f'{model_type}_metrics', {})
                if metrics.get('success', False):
                    mape_val = metrics.get('mape', np.nan)
                    rmse_val = metrics.get('rmse', np.nan)
                    if mape_val < np.inf:
                        mape_values.append(mape_val)
                    if rmse_val < np.inf:
                        rmse_values.append(rmse_val)
                
                # Count seasonality patterns
                stat_features = result.get('statistical_features', {})
                if stat_features.get('has_weekly_seasonality', False):
                    weekly_count += 1
                if stat_features.get('has_monthly_seasonality', False):
                    monthly_count += 1
                if stat_features.get('has_yearly_seasonality', False):
                    yearly_count += 1
            
            if mape_values:
                print(f"  Average MAPE: {np.mean(mape_values):.2f}%")
                print(f"  Median MAPE: {np.median(mape_values):.2f}%")
            if rmse_values:
                print(f"  Average RMSE: {np.mean(rmse_values):.2f}")
                print(f"  Median RMSE: {np.median(rmse_values):.2f}")
            print(f"  Total series: {len(series_list)}")
            
            # Seasonality statistics
            print(f"  Seasonality Detection:")
            print(f"    Weekly: {weekly_count}/{len(series_list)} series ({weekly_count/len(series_list)*100:.1f}%)")
            print(f"    Monthly: {monthly_count}/{len(series_list)} series ({monthly_count/len(series_list)*100:.1f}%)")
            print(f"    Yearly: {yearly_count}/{len(series_list)} series ({yearly_count/len(series_list)*100:.1f}%)")
            
            # Print recommended Prophet parameters if available
            if model_type == 'prophet' and prophet_params is not None and category_name in prophet_params:
                params = prophet_params[category_name]
                if params and isinstance(params, dict):
                    print(f"\n  Recommended Prophet Parameters:")
                    print(f"    {{")
                    # Sort keys but handle add_seasonalities specially to ensure all are printed
                    sorted_keys = sorted([k for k in params.keys() if k != 'add_seasonalities'])
                    # Print add_seasonalities first if it exists (to make it more visible)
                    if 'add_seasonalities' in params and params['add_seasonalities']:
                        print(f"      'add_seasonalities': [")
                        # Ensure we iterate through ALL seasonalities in the list
                        for seasonality in params['add_seasonalities']:
                            if isinstance(seasonality, dict) and 'name' in seasonality:
                                print(f"          {{")
                                print(f"              'name': '{seasonality['name']}',")
                                print(f"              'seasonal_periods': {seasonality['seasonal_periods']},")
                                print(f"              'fourier_order': {seasonality['fourier_order']}")
                                print(f"          }},")
                        print(f"      ],")
                    # Print other parameters
                    for key in sorted_keys:
                        value = params[key]
                        if isinstance(value, bool):
                            print(f"      '{key}': {value},")
                        elif isinstance(value, str):
                            print(f"      '{key}': '{value}',")
                        else:
                            print(f"      '{key}': {value},")
                    print(f"    }}")
            print()


In [ ]:
# Plot Prophet classifications (with recommended parameters shown in summary)
print("="*60)
print("PROPHET CLASSIFICATIONS - Historical Data by Category")
print("="*60)
plot_series_by_category(evaluation_results, model_type='prophet', max_series_per_plot=MAX_SERIES_PER_PLOT, 
                        prophet_params=prophet_params_for_plotting)


In [ ]:
# Plot ARIMA classifications
print("="*60)
print("ARIMA CLASSIFICATIONS - Historical Data by Category")
print("="*60)
plot_series_by_category(evaluation_results, model_type='arima', max_series_per_plot=MAX_SERIES_PER_PLOT)


## 9. Prophet Parameter Helper Function

Helper function for generating Prophet parameters (used internally by plotting function).


In [ ]:
def get_recommended_prophet_params(category, stat_features=None):
    """Get recommended Prophet parameters based on category and detected seasonality.
    
    Args:
        category: Predictability category ('Predictable', 'Low Predictability', etc.)
        stat_features: Dictionary of statistical features including seasonality detection
    """
    
    # Detect seasonality from features if provided
    has_weekly = False
    has_monthly = False
    has_yearly = False
    
    if stat_features:
        has_weekly = stat_features.get('has_weekly_seasonality', False)
        has_monthly = stat_features.get('has_monthly_seasonality', False)
        has_yearly = stat_features.get('has_yearly_seasonality', False)
    
    # Base parameters - enable seasonality based on detection
    base_params = {
        'yearly_seasonality': has_yearly,  # Enable if detected
        'weekly_seasonality': has_weekly,  # Enable if detected
        'daily_seasonality': False,  # Usually not needed for daily aggregated data
        'seasonality_mode': 'additive',
    }
    
    # Category-specific recommendations
    if category == 'Predictable':
        # Predictable series: balanced flexibility (combines former Highly + Moderately Predictable)
        # Use moderate flexibility to handle both strong and moderate patterns
        return {
            **base_params,
            'changepoint_prior_scale': 0.5,  # Moderate flexibility
            'seasonality_prior_scale': 10.0,  # Strong seasonality if present
            'holidays_prior_scale': 10.0,
            'mcmc_samples': 0,  # No MCMC (faster, deterministic)
            'interval_width': 0.80,
            'uncertainty_samples': 1000,
        }
    
    elif category == 'Low Predictability':
        # Low predictability: more flexible model to capture complex patterns
        # Lower changepoint_prior_scale = more changepoints = more flexible
        return {
            **base_params,
            'changepoint_prior_scale': 0.05,  # More flexible, more changepoints
            'seasonality_prior_scale': 0.01,  # Allow seasonality to adapt more
            'holidays_prior_scale': 10.0,
            'mcmc_samples': 0,
            'interval_width': 0.95,  # Wider intervals (less confident)
            'uncertainty_samples': 1000,
        }
    
    else:  # Not Suitable
        # For not suitable series, provide conservative defaults
        # but note that forecasting may not be recommended
        return {
            **base_params,
            'changepoint_prior_scale': 0.1,  # Very flexible
            'seasonality_prior_scale': 0.01,
            'holidays_prior_scale': 10.0,
            'mcmc_samples': 0,
            'interval_width': 0.95,
            'uncertainty_samples': 1000,
        }

# Helper function for generating Prophet parameters
# Note: Parameters are displayed when plotting by category (Section 8)
# This function is kept for reference and potential future use


## Summary and Recommendations

Final summary of the analysis with recommendations for model selection.


In [ ]:
print("="*60)
print("FINAL SUMMARY AND RECOMMENDATIONS")
print("="*60)

print(f"\nTotal Series Analyzed: {len(evaluation_results)}")
print(f"Query Selector: {SELECTOR}")
print(f"History Period: {HISTORY_DAYS} days")

# Prophet summary
prophet_predictable = sum(1 for r in evaluation_results if r.get('prophet_classification') == 'Predictable')
prophet_low = sum(1 for r in evaluation_results if r.get('prophet_classification') == 'Low Predictability')
prophet_not = sum(1 for r in evaluation_results if r.get('prophet_classification') == 'Not Suitable')

print(f"\nProphet Model Suitability:")
print(f"  Predictable: {prophet_predictable} ({prophet_predictable/len(evaluation_results)*100:.1f}%)")
print(f"  Low Predictability: {prophet_low} ({prophet_low/len(evaluation_results)*100:.1f}%)")
print(f"  Not Suitable: {prophet_not} ({prophet_not/len(evaluation_results)*100:.1f}%)")

# ARIMA summary
arima_predictable = sum(1 for r in evaluation_results if r.get('arima_classification') == 'Predictable')
arima_low = sum(1 for r in evaluation_results if r.get('arima_classification') == 'Low Predictability')
arima_not = sum(1 for r in evaluation_results if r.get('arima_classification') == 'Not Suitable')

print(f"\nARIMA Model Suitability:")
print(f"  Predictable: {arima_predictable} ({arima_predictable/len(evaluation_results)*100:.1f}%)")
print(f"  Low Predictability: {arima_low} ({arima_low/len(evaluation_results)*100:.1f}%)")
print(f"  Not Suitable: {arima_not} ({arima_not/len(evaluation_results)*100:.1f}%)")

# Recommendations
print(f"\nRecommendations:")
print(f"  - Use Prophet for series classified as 'Predictable'")
print(f"  - Use ARIMA for series where ARIMA shows better performance (lower MAPE)")
print(f"  - Consider alternative approaches for 'Not Suitable' series")
print(f"  - Review 'Low Predictability' series - may need feature engineering or different models")

# Find series where one model is clearly better
better_prophet = 0
better_arima = 0
equal = 0

for result in evaluation_results:
    prophet_metrics = result.get('prophet_metrics', {})
    arima_metrics = result.get('arima_metrics', {})
    
    if prophet_metrics.get('success', False) and arima_metrics.get('success', False):
        prophet_mape = prophet_metrics.get('mape', np.inf)
        arima_mape = arima_metrics.get('mape', np.inf)
        
        if prophet_mape < np.inf and arima_mape < np.inf:
            diff = abs(prophet_mape - arima_mape)
            if diff < 1.0:  # Within 1% - consider equal
                equal += 1
            elif prophet_mape < arima_mape:
                better_prophet += 1
            else:
                better_arima += 1

print(f"\nModel Comparison (where both models succeeded):")
print(f"  Prophet performs better: {better_prophet}")
print(f"  ARIMA performs better: {better_arima}")
print(f"  Similar performance: {equal}")

print("\n" + "="*60)
